In [2]:
# JN1ORP 20230510
#
# 事前準備
#
# Google Drive に pota フォルダーを作成
# hunter_pars.csv：POTAサイトからハンター情報をダウンロードし、potaフォルダーにコピー
#

# ライブラリのインポート
import os
import shutil
import folium
import pandas as pd

# Googleドライブのマウント
from google.colab import drive
drive.mount("/content/drive")

print('Begin')

# maps フォルダの再作成
os.makedirs("/content/drive/My Drive/pota/maps", exist_ok=True) # 無い時にrmtreeのエラーを防ぐ
shutil.rmtree("/content/drive/My Drive/pota/maps")
os.makedirs("/content/drive/My Drive/pota/maps", exist_ok=True)

# ハント情報の読み込み 対象を日本に絞る
hunter=pd.read_csv("/content/drive/My Drive/pota/hunter_parks.csv")
hunter_jp = hunter.query("Reference.str.contains('JP-', regex=True)", engine="python").reset_index()
hunter_jp.head()

# 公園情報の読み込み
parks = pd.read_json("https://tyjcode.github.io/y001/parks-latest.json")
parks.head()

# ハント情報に公園情報をマージ
hunter_map = pd.merge(hunter_jp, parks, on="Reference")
hunter_map.head()

# 都道府県でグルーピング 件数をカウント
hunter_pref_group = hunter_jp.groupby("Location").count().iloc[:,:1].reset_index()
hunter_pref_group.head()

# ハントしていない公園を抽出 (非アクティブを含む)
non_hunter = pd.merge(parks, hunter_jp, on="Reference", how="outer", indicator=True)
non_hunter_map = non_hunter[non_hunter["_merge"] == "left_only"].iloc[:,:9]
non_hunter_map.head()

#
# 全国地図にプロット
#
map = folium.Map(location=[35.7503, 139.523], zoom_start=6)

# ハントしてない公園
for i, r in non_hunter_map.iterrows():
  name_pop = folium.Popup(f'{r["Reference"]}{r["日本語名"]}', max_width=1000)
  folium.Marker(location=[r["latitude"], r["longitude"]], popup=name_pop,icon=folium.Icon(color='red')).add_to(map)

# ハントした公園
for i, r in hunter_map.iterrows():
  name_pop = folium.Popup(f'{r["Reference"]}{r["日本語名"]}{r["QSOs"]}', max_width=1000)
  folium.Marker(location=[r["latitude"], r["longitude"]], popup=name_pop).add_to(map)

# 地図の作成
map.save("/content/drive/My Drive/pota/maps/map_parks.html")

#
# ハントした都道府県別に地図を作成
#

for j, h in hunter_pref_group.iterrows():

  # 都道府県毎に抽出
  non_hunter_map_pref = non_hunter_map.query('location=="{}"'.format(h["Location"])).reset_index()
  hunter_map_pref = hunter_map.query('Location=="{}"'.format(h["Location"])).reset_index()

  # 最初の公園を中心に地図を作成

  # 小笠原のデーターが分離したので、対応 20250224
  # hunter_map_pref が空かどうかを確認してから要素にアクセスする
  if not hunter_map_pref.empty:
    map2 = folium.Map(location=[hunter_map_pref.loc[0,"latitude"], hunter_map_pref.loc[0,"longitude"]], zoom_start=10)
  else:
    # hunter_map_pref が空の場合の処理、例：
    print(f"都道府県 {h['Location']} のデータが見つかりませんでした。地図の作成をスキップします。")
    continue  # 次の反復処理にスキップする

  # ... (残りのコードは同じまま) ...

  # 最初の公園を中心に地図を作成
  map2 = folium.Map(location=[hunter_map_pref.loc[0,"latitude"], hunter_map_pref.loc[0,"longitude"]], zoom_start=10)

  # ハントしてない公園
  for i, r in non_hunter_map_pref.iterrows():
    name_pop = folium.Popup(f'{r["Reference"]}{r["日本語名"]}', max_width=1000)
    folium.Marker(location=[r["latitude"], r["longitude"]], popup=name_pop,icon=folium.Icon(color="red")).add_to(map2)

  # ハントした公園
  for i, r in hunter_map_pref.iterrows():
    name_pop = folium.Popup(f'{r["Reference"]}{r["日本語名"]}{r["QSOs"]}', max_width=1000)
    folium.Marker(location=[r["latitude"], r["longitude"]], popup=name_pop).add_to(map2)

  # 地図の作成
  map2.save(f'/content/drive/My Drive/pota/maps/map_parks2_{h["Location"]}_{h["index"]}.html')

print('End')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Begin
都道府県 Ogasawara のデータが見つかりませんでした。地図の作成をスキップします。
End
